# Reconstruction of the Invariant Mass of the Z boson in C++

In this analysis, we are focusing on identifying the decay of the [Z bosons](https://en.wikipedia.org/wiki/W_and_Z_bosons), a fundamental particle responsible for mediating the weak nuclear force. Let's get started!

<CENTER>
    <a href="http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html" class="icons"><img src="../../images/ATLASOD.gif" style="width:40%"></a>
</CENTER>

## What is the Z Boson?
The Z boson is one of the mediators of the weak force, which is responsible for processes such as [beta decay](https://en.wikipedia.org/wiki/Beta_decay) in atomic nuclei. It interacts with all known fermions (quarks and leptons), but unlike the W boson, it does not change the type (flavor) of particle it interacts with. The Z boson couples to both [left-handed and right-handed](https://en.wikipedia.org/wiki/Chirality_(physics)) particles, making its behavior distinct from the charged W boson.

Since the Z boson is electrically neutral, its decay products must have balanced charges. The decays of the Z boson into leptons (electrons, muons, and taus) are particularly useful for experimental studies because these particles can be precisely measured in detectors, giving a clear signature of the Z boson's presence.

## The Decay of the Z Boson

The Z boson decays rapidly due to its high mass, with a mean lifetime of around 3 × 10$^{-25}$ seconds. Its decay channels include hadrons (quarks) and leptons, but in this analysis, we are particularly interested in the lepton channels because they produce clean final states that are easier to measure.

In this notebook, we are looking for events where the Z boson decays to two leptons of the same flavor but opposite charge (e.g., Z → e$^+$e$^-$ or Z → μ$^+$μ$^-$). Showed in the image on the left.

<table style="width:100%; text-align:center;">
  <tr>
    <td><img src="../../images/Z_ElectronPositron.png" style="width:33%"></td>
  </tr>
</table>


The aim of this analysis is to plot the Z peak, which refers to the resonant mass of the Z boson observed in the invariant mass distribution of the two leptons. By plotting the invariant mass of the two-lepton system, we expect to see a peak around 91 GeV/c², which corresponds to the mass of the Z boson.

## Running a Jupyter notebook
A Jupyter notebook consists of cell blocks, each containing lines of Python code. Each cell can be run independently of each other, yielding respective outputs below the cells. Conventionally,cells are run in order from top to bottom.

- To run the whole notebook, in the top menu click Cell $\to$ Run All.
- To propagate a change you've made to a piece of code, click Cell $\to$ Run All Below.
- You can also run a single code cell, by clicking Cell $\to$ Run Cells, or using the keyboard shortcut Shift+Enter.

For more information, refer to [How To Use Jupyter Notebooks](https://www.codecademy.com/article/how-to-use-jupyter-notebooks).

By the end of this notebook you will be able to:
1. Learn to process large data sets using cuts
2. Understand some general principles of a particle physics analysis
3. Discover the Z boson!

## The analysis

The following section details the steps taken to analyze the Z boson decays from our simulated data sample. The analysis revolves around reading, processing, and visualizing events where the Z boson decays into leptons.

We need to include some standard C++ and ROOT libraries

In [1]:
#include <iostream>
#include <string>
#include <stdio.h>

In order to activate the interactive visualisation of the histogram that is later created we can use the JSROOT magic:

In [2]:
%jsroot on

Because we would like to use more than one ROOT input file, the best option is to use a TChain object. This allows to "chain" several samples into a single structure that we can later loop over

In [3]:
TChain *dataset = new TChain("mini");

This following input is read directly from the same storage - so it is faster.

In [4]:
dataset->Add("root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2016-07-29/MC/mc_147770.Zee.root"); // 8TeV sample

Now we're going to extract the lepton variables

In [5]:
const int vs = 5;

UInt_t  lepton_n = -1,
        lepton_charge[vs], //electrical charge of the lepton
        lepton_type[vs];   //flavour of the lepton

Float_t lepton_pt[vs],
        lepton_eta[vs],
        lepton_phi[vs],
        lepton_E[vs];

Here we're filling the variables defined above with the content of those inside the input ntuple

In [6]:
dataset->SetBranchAddress("lep_n",      &lepton_n);
dataset->SetBranchAddress("lep_charge", &lepton_charge);
dataset->SetBranchAddress("lep_type",   &lepton_type);
dataset->SetBranchAddress("lep_pt",     &lepton_pt);
dataset->SetBranchAddress("lep_eta",    &lepton_eta);
dataset->SetBranchAddress("lep_phi",    &lepton_phi);
dataset->SetBranchAddress("lep_E",      &lepton_E);

Now we will create a histogram specifying its title and axes titles. Moreover we define that it consists of 30 bins which have a range from 40 to 140. Additionaly, we set the colour of the integral blue.

In [7]:
TH1F *hist = new TH1F("variable","Mass of the Z boson; mass [GeV]; events",30,40,140);
hist->SetFillColor(kBlue);

In the following lines we can say how much data we want to analyze and we also ask some informations about the analysis itself. The results are given out under the box.

In [8]:
int nentries, nbytes, i;
nentries = (Int_t)dataset->GetEntries();

// IMPORTANT: fraction events we want to run
float fraction_events = 0.1;
int events_to_run = nentries*fraction_events;

std::cout << "Total # events = "  << nentries
          << ". Events to run = " << events_to_run
          << " corresponding to " << fraction_events*100
          << "% of total events!" << std::endl;

Total # events = 7500000. Events to run = 750000 corresponding to 10% of total events!


To fill the histogram, we loop over each event in the TTree, applying several cuts to select the events of interest. The goal is to isolate Z boson decays into two leptons:

- **Cut #1: At least two leptons**. This ensures that the event contains two leptons, which are necessary for reconstructing the Z boson mass.
- **Cut #2: Opposite charge leptons**. Since the Z boson is neutral, it decays into a lepton and its antiparticle (opposite charge).
- **Cut #3: Same lepton family**. We select events where both leptons are either electrons (Z → e$^+$e$^-$) or muons (Z → μ$^+$μ$^-$).

For each event passing the cuts, we calculate the invariant mass using the four-momenta of the leptons and fill the histogram.

In [9]:
TLorentzVector l1;
TLorentzVector l2;
TLorentzVector invmass;

for (i = 0; i < events_to_run; i++)
{
    nbytes = dataset->GetEntry(i);
    
    // Cut #1: At least 2 leptons
    if(lepton_n == 2)
    {
        // Cut #2: Leptons with opposite charge
        if(lepton_charge[0] != lepton_charge[1])
        {
            // Cut #3: Leptons of the same family (2 electrons or 2 muons)
            if(lepton_type[0] == lepton_type[1])
            {
                l1.SetPtEtaPhiE(lepton_pt[0], lepton_eta[0], lepton_phi[0], lepton_E[0]);
                l2.SetPtEtaPhiE(lepton_pt[1], lepton_eta[1], lepton_phi[1], lepton_E[1]);
                // The default value in the root file is in MeV, so, we divide by 1000 to get it in GeV
                invmass = l1 + l2;
                hist->Fill(invmass.M()/1000.);
            }
        }
    }
}

Once the histogram is filled, we can visualize it. We use the `Draw()` method to display the histogram on the canvas. We render the canvas to display the results of our analysis, showing the invariant mass distribution of the Z boson decay products.

In [10]:
TCanvas *canvas = new TCanvas("c","c",10,10,700,700);
hist->Draw();
canvas->Draw();

## Understanding the plot

The plot above represents the invariant mass distribution of two leptons (electrons or muons) produced in the decay of the Z boson. This histogram was filled by calculating the invariant mass of pairs of leptons (with opposite charge and same flavor) from the dataset, with 30 bins covering a mass range from 40 to 140 GeV.

This is what we are looking at in the plot:

- **Z Peak at ~91 GeV**: The prominent peak near 91 GeV corresponds to the mass of the Z boson, which is well-known to be approximately 91 GeV/c$^2$. This peak is the result of many events where a Z boson was produced and decayed into a lepton-antilepton pair (Z → e$^+$e$^-$). The number of entries (37415) reflects the number of such events detected and analyzed in this dataset.
- **Mean and Standard Deviation**:
    - **Mean Mass**: The mean of the distribution is 87.94 GeV, close to the expected mass of the Z boson.
    - **Standard Deviation** (7.896 GeV): This value reflects the width of the mass distribution. The spread around the peak is due to experimental resolution, detector effects, and the natural width of the Z boson (~2.5 GeV). The broader standard deviation may also reflect the complexity of the event reconstruction and the simplicity of this analysis.
- **Low and High-Mass Tails**: The tails of the distribution, extending below 80 GeV and above 100 GeV, correspond to background events. These can arise from misidentified or non-resonant processes that do not involve the Z boson but still produce lepton pairs. Events outside the Z boson peak range may come from processes such as Drell-Yan or other electroweak backgrounds.
- **Y-Axis** (Events): The y-axis shows the number of events in each bin, which indicates how many events with a given invariant mass were observed. The highest point of the peak corresponds to the most frequent mass value for Z boson decays, which is around 91 GeV.